In [1]:
import OpenOPC as OP
import time
import threading as th
#import wx
import datetime
import serial
import pywintypes


In [2]:
pywintypes.datetime=pywintypes.TimeType

In [3]:
opc=OP.client()

In [4]:
servidor=opc.servers()
print(servidor)

['ABB.AC800MC_OpcDaServer.3', 'Matrikon.OPC.Simulation.1']


In [6]:
opc.connect('ABB.AC800MC_OpcDaServer.3','192.168.99.10')

In [25]:
opc.read("Applications.Application_1.Program2.bool1")

(True, 'Good', '2023-03-17 10:18:18.519000+00:00')

In [26]:
opc.read("Applications.Application_1.b1")

(True, 'Good', '2023-03-17 10:18:44.714000+00:00')

### Definicion de variables globales

In [16]:
opcserver = "ABB.AC800MC_OpcDaSarver.3"
mainTagname = "Applications.CAPEM"
#lista de comandos:
#cerrar: finaliza el programa
close="5"
#iniciar: inicia la medicion
start= "1"
#detener: detiene la medicion
stop="2"
#envia una nueva configuracion
config="0"
#Chequea el estado de la aplicacion
status="4"
#Diccionario para mensajes luego de los comandos
comdic={config:"Cargando configuraciones", start:"Mediciones iniciadas", stop:"Mediciones detenidas", close:"Cerrando programa", status: "Estado de sistema:"}
comando=config
comprev=comando
variables=[]
device=serial.Serial()

### Funciones


In [ ]:
## Funcion que carga configuraciones
from configuracion import configurar

In [ ]:
##Funcion que chequea estado
def check_estado():
    "Funcion que chequea estado del sistema"
    global comprev, device, comdic
    
    if device.isOpen():
        dsp="Dispositivo conectado\n"
    else:
        dsp="Dispositivo desconectado\n"
        
    cmd=comdic[comprev]+"\n"
    
    estado="\n\n -Dispositivo de medición: "+dsp+" -Comando actual: "+cmd
    
    return estado

In [ ]:
## Funcion que permite ingresar comandos desde consola
def comandos():
    "Funcion que permite ingresar comandos desde consola"
    global comando, close, comdic, comprev
    while comando!=close:
        comando1 =input("\ningrese comando:\n  0. Cargar nueva configuracion\n  1. Iniciar medicion\n  2. Detener medicion\n  4. Estado\n  5. Salir\n")
        if comando1 in comdic:
            comprev=comando
            comando=comando1
            print(comdic[comando])
        else:
            print("el comando {} no se reconoce".format(comando1))

In [ ]:
def medir(tiempo=3.0,parametro=0):
    "Rutina que toma las mediciones de CAPEM y del instrumento"
    global comando,start, variables, device
    #Acá va la funcion que mide las variables de CAPEM y del instrumento 
    escritura=0
    while comando == start:
        escritura+=1
        wr=bytes("121","utf-8")
        device.write(wr)
        time.sleep(0.5)
        resp=device.read(3)
        print(resp.decode('utf-8'))
        #print(datetime.datetime.now())
        time.sleep(tiempo)
    #print ("medicion finalizada")
    

### Rutina principal

In [22]:
##### Concectar puerto serie

if device.port==None:
    device=serial.Serial(port="COM3",baudrate=19200,bytesize=8, parity="N", stopbits=1, timeout=10)
elif device.isOpen()==False:
    device.open()
##### Thread 2, Busca comandos
th2=th.Thread(target=comandos,name="th2")
th2.start()

## Loop principal
while comando != close:
    if comando == status:
        sts=check_estado()
        print(sts)
        comando=comprev
    if comando == start:
        medir()
           
    if comando == config:
        configurar("/config.txt")
        comando=stop
        

print("Programa finalizado")
if device.isOpen() == True:
    device.close()
    #print("ajam")
    #comando1=input("ingrese comando:\n  0. Cargar nueva configuracion\n  1. Iniciar medicion\n  2. Detener medicion\n  5. Salir\n")
    
    

    

NameError: name 'comandos' is not defined

In [ ]:
device.close()

In [ ]:
medir()